In [ ]:
!nvidia-smi

Fri Mar 21 18:34:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             49W /  400W |       1MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# Finetuning T5 for name correction
https://www.kdnuggets.com/how-to-fine-tune-t5-for-question-answering-tasks-with-hugging-face-transformers

In [ ]:
!pip install -q python-dotenv datasets pandas transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 35.7 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
load_dotenv()
token = os.environ.get("HUGGINGFACE_AUTH_TOKEN")
login(token = token)

In [ ]:
from datasets import load_dataset

# Load dataset from Hugging Face Hub
hf_dataset = load_dataset("juanluisrto/names-es")

# Ensure it's correctly loaded
print(hf_dataset)

README.md:   0%|          | 0.00/594 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/287M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/287M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/287M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/95.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21648292 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2405366 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2672629 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', '__index_level_0__'],
        num_rows: 21648292
    })
    validation: Dataset({
        features: ['input', 'output', '__index_level_0__'],
        num_rows: 2405366
    })
    test: Dataset({
        features: ['input', 'output', '__index_level_0__'],
        num_rows: 2672629
    })
})


### Finding unknown characters to add to the tokenizer

There might be some characters particular to spanish which the tokenizer has not been trained on.
Let's double check

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Initialize the T5 tokenizer and model (T5-small in this case)
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


In [ ]:
def encode_decode(text):
    return tokenizer.decode(tokenizer(text)["input_ids"], skip_special_tokens=True)

In [ ]:
# We test with some common characters in spanish / catalan
text = "áàéèíìóòúüùñ"
text += text.upper() # we also add them in uppercase
decoded = encode_decode(text)
print(text, decoded)
unknown_characters = set(text) - set(decoded)
print("characters labelled as <unk>:", unknown_characters)

áàéèíìóòúüùñÁÀÉÈÍÌÓÒÚÜÙÑ áàéèóüùÉÜ
characters labelled as <unk>: {'Ú', 'Í', 'Ì', 'ñ', 'À', 'ì', 'È', 'ò', 'Ù', 'Á', 'ú', 'í', 'Ó', 'Ò', 'Ñ'}


#### Finding unknown characters in the dataset.
Lets also run a sample of all the names and surnames through the tokenizer just in case we are missing any

In [ ]:
clean_df = pd.read_parquet("./df_es_clean.parquet")
names_and_surnames = clean_df.first_name.unique().tolist() + clean_df.last_name.unique().tolist()
len(names_and_surnames)

2660843

In [ ]:
import random

names_and_surnames_sample = random.sample(names_and_surnames, 30000)
#names_and_surnames

In [ ]:
from tqdm import tqdm
from collections import Counter

problematic_chars = []

for text in tqdm(names_and_surnames_sample):
    decoded = encode_decode(text)
    if decoded != text:
        for c in text:
            if c not in decoded:
                problematic_chars.append(c)

counter = Counter(problematic_chars)
print("Problematic characters:", counter.most_common(10))

100%|██████████| 30000/30000 [00:12<00:00, 2424.45it/s]

Problematic characters: [('ñ', 1730), ('í', 1157), ('ú', 105), ('Á', 91), ('ا', 17), ('ò', 15), ('ň', 14), ('م', 13), ('я', 13), ('ي', 12)]


There are no other important characters we need to add a part from the ones already identified.

In [ ]:
unknown_characters

{'À', 'Á', 'È', 'Ì', 'Í', 'Ñ', 'Ò', 'Ó', 'Ù', 'Ú', 'ì', 'í', 'ñ', 'ò', 'ú'}

In [ ]:
tokenizer.add_tokens(list(unknown_characters))
model.resize_token_embeddings(len(tokenizer))

Embedding(32115, 512)

In [ ]:

import multiprocessing
num_cpus = multiprocessing.cpu_count()
print(f"{num_cpus} CPUs available")

def preprocess_function(examples):
    inputs = examples["input"]
    targets = examples["output"]
    model_inputs = tokenizer(inputs, max_length=64, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=64, truncation=True, padding='max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the datasets
tokenized_datasets = hf_dataset.map(preprocess_function, batched=True, num_proc=num_cpus)

12 CPUs available


Map (num_proc=12):   0%|          | 0/21648292 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/2405366 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/2672629 [00:00<?, ? examples/s]

# Training

In [37]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    print(eval_pred)
    predictions = logits.argmax(axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="steps",
    save_steps=1500,        # Save a checkpoint every 1500 steps
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    report_to=["tensorboard"],
    push_to_hub=True,              # Enable pushing to Hugging Face Hub after training
    hub_model_id="juanluisrto/t5-small-namefixer-es",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,

)


print(trainer.args.device)  # Should show "cuda"

cuda:0


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
trainer.train(
    #resume_from_checkpoint="./results/checkpoint-7000"
    )

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


Buffered data was truncated after reaching the output size limit.

In [34]:
tokenizer.push_to_hub("juanluisrto/t5-small-namefixer-es")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/juanluisrto/t5-small-namefixer-es/commit/c3af040a50696fc221b2d6542eda2abe6f768c3a', commit_message='Upload tokenizer', commit_description='', oid='c3af040a50696fc221b2d6542eda2abe6f768c3a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/juanluisrto/t5-small-namefixer-es', endpoint='https://huggingface.co', repo_type='model', repo_id='juanluisrto/t5-small-namefixer-es'), pr_revision=None, pr_num=None)

In [63]:
sample = tokenized_datasets["test"].select(range(1000))
metrics = trainer.evaluate(eval_dataset=sample)
print(metrics)

{'eval_loss': 0.006400068290531635, 'eval_model_preparation_time': 0.0042, 'eval_runtime': 2.8981, 'eval_samples_per_second': 345.057, 'eval_steps_per_second': 43.132}


In [64]:
trainer.push_to_hub("juanluisrto/t5-small-namefixer-es")  # Pushes the final model and training artifacts

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/juanluisrto/t5-small-namefixer-es/commit/ce9751348f9886f19c1b606bad315fc099b03529', commit_message='juanluisrto/t5-small-namefixer-es', commit_description='', oid='ce9751348f9886f19c1b606bad315fc099b03529', pr_url=None, repo_url=RepoUrl('https://huggingface.co/juanluisrto/t5-small-namefixer-es', endpoint='https://huggingface.co', repo_type='model', repo_id='juanluisrto/t5-small-namefixer-es'), pr_revision=None, pr_num=None)

In [83]:

import re
chars = "ÀÁÈÌÍÑÒÓÙÚìíñòú"
pattern = re.compile(rf"(?<=\w)\s+([{chars}])|([{chars}])\s+(?=\w)")

def fix_unknown_chars(text: str) -> str:
    return pattern.sub(lambda m: m.group(1) or m.group(2), text)



In [82]:

input_text = "María José García Arenas Muñoz"

device = "cuda"

# Move model to device
model.to(device)

# Tokenize input and move to device
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# Generate the answer using the model
output_ids = model.generate(input_ids)

# Decode the generated tokens to get the predicted answer
predicted_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print(f"Input text: {input_text}")
# Print the predicted answer
print(f"Predicted answer: {predicted_answer}")
clean_answer = fix_unknown_chars(predicted_answer)
print(f"Clean answer: {clean_answer}")


print("")
print("=======")
for output in output_ids[0]:
  print(str(output.item()).rjust(7, " "), tokenizer.decode(output))

Input text: María José García Arenas Muñoz
Predicted answer: Mar í a José|Garc í a Arenas Mu ñ oz|F
Clean answer: Marí a José|Garcí a Arenas Muñ oz|F

      0 <pad>
   1571 Mar
  32111 í
      3 
      9 a
  26816 José
   9175 |
  21846 Gar
     75 c
  32111 í
      3 
      9 a
  14904 Arena
      7 s
   4159 Mu
  32103 ñ
      3 
     32 o
    172 z
   9175 |
    371 F
